# 02 - Prompt Engineering

**Master the art of effective prompting** to get better, more reliable results from LLMs.

## Learning Objectives

By the end of this notebook, you will:
- Understand zero-shot vs few-shot prompting
- Write effective system prompts
- Use chain-of-thought prompting for reasoning
- Create reusable prompt templates

## Table of Contents

1. [Why Prompt Engineering Matters](#why)
2. [Zero-Shot Prompting](#zero-shot)
3. [Few-Shot Prompting](#few-shot)
4. [System Prompts & Roles](#system-prompts)
5. [Chain-of-Thought Prompting](#cot)
6. [Prompt Templates](#templates)
7. [Exercises](#exercises)
8. [Checkpoint](#checkpoint)

In [ ]:
# GUIDED: Setup
import os
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(Path.cwd().parent / ".env")

client = OpenAI()

def ask(prompt, system=None, temperature=0.7):
    """Helper function to make API calls."""
    messages = []
    if system:
        messages.append({"role": "system", "content": system})
    messages.append({"role": "user", "content": prompt})
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=temperature
    )
    return response.choices[0].message.content

print("Setup complete!")

---
## 1. Why Prompt Engineering Matters <a id='why'></a>

The same model can give vastly different results based on how you ask.

### Bad Prompt vs Good Prompt

In [ ]:
# GUIDED: Compare bad and good prompts

# Bad prompt - vague
bad_prompt = "Tell me about Python"
print("Bad Prompt:")
print(f"  '{bad_prompt}'")
print(f"  Response: {ask(bad_prompt)[:200]}...\n")

# Good prompt - specific
good_prompt = """Explain Python's list comprehension feature.

Include:
1. A simple example
2. When to use it
3. A comparison with regular for loops

Keep the response under 150 words."""

print("Good Prompt:")
print(f"  (structured with clear requirements)")
print(f"  Response: {ask(good_prompt)}")

### Key Principles

1. **Be Specific**: Tell the model exactly what you want
2. **Provide Context**: Give background information
3. **Set Constraints**: Specify format, length, style
4. **Use Examples**: Show the model what you expect

---
## 2. Zero-Shot Prompting <a id='zero-shot'></a>

**Zero-shot** means asking the model to perform a task without any examples.

The model relies on its training to understand what you want.

In [ ]:
# GUIDED: Zero-shot examples

# Classification
prompt1 = """Classify the sentiment of this review as POSITIVE, NEGATIVE, or NEUTRAL:

Review: "The product arrived on time but the quality was disappointing."

Sentiment:"""

print("Classification (zero-shot):")
print(ask(prompt1, temperature=0))

# Extraction
prompt2 = """Extract the following information from this text:
- Name
- Company
- Role

Text: "Hi, I'm Sarah Chen and I work as a Senior Engineer at TechCorp."

Output as JSON:"""

print("\nExtraction (zero-shot):")
print(ask(prompt2, temperature=0))

### When Zero-Shot Works Well

- Common tasks (summarization, translation, Q&A)
- Clear instructions
- Well-defined output format

---
## 3. Few-Shot Prompting <a id='few-shot'></a>

**Few-shot** means providing examples of the task before asking the model to perform it.

This helps the model understand the pattern you want.

In [ ]:
# GUIDED: Few-shot prompting

few_shot_prompt = """Convert natural language to SQL queries.

Example 1:
Input: Show all users
Output: SELECT * FROM users;

Example 2:
Input: Find users named John
Output: SELECT * FROM users WHERE name = 'John';

Example 3:
Input: Count total orders
Output: SELECT COUNT(*) FROM orders;

Now convert:
Input: Find all products with price greater than 100
Output:"""

print("Few-Shot SQL Generation:")
print(ask(few_shot_prompt, temperature=0))

In [ ]:
# GUIDED: Few-shot for custom formats

custom_format_prompt = """Generate product descriptions in our brand voice.

Example 1:
Product: Wireless headphones
Description: Experience audio freedom with our sleek wireless headphones. 
Crystal-clear sound meets all-day comfort. Your soundtrack, untethered.

Example 2:
Product: Smart water bottle
Description: Hydration meets innovation with our smart water bottle.
Track your intake, sync with your fitness goals. Drink smarter, live better.

Now write:
Product: Portable laptop stand
Description:"""

print("Custom Format Generation:")
print(ask(custom_format_prompt))

### Few-Shot Best Practices

1. **Use 2-5 examples** - enough to show the pattern
2. **Diverse examples** - cover different cases
3. **Consistent format** - same structure in all examples
4. **Representative examples** - similar to actual use cases

---
## 4. System Prompts & Roles <a id='system-prompts'></a>

**System prompts** set the behavior and personality of the model.

They're persistent instructions that apply to the entire conversation.

In [ ]:
# GUIDED: Different system prompts = different responses

question = "What should I do if my code has a bug?"

# Casual assistant
casual_system = "You are a friendly, casual coding buddy. Use informal language and emojis."

print("Casual Assistant:")
print(ask(question, system=casual_system))
print("\n" + "="*50 + "\n")

# Professional consultant
professional_system = """You are a senior software engineering consultant.
Provide structured, professional advice.
Use bullet points and technical terminology."""

print("Professional Consultant:")
print(ask(question, system=professional_system))

In [ ]:
# GUIDED: Role-based system prompts

expert_system = """You are an expert Python developer with 15 years of experience.

Your expertise includes:
- Web development (Django, FastAPI)
- Data science (pandas, scikit-learn)
- DevOps (Docker, Kubernetes)

When answering questions:
1. Consider best practices and common pitfalls
2. Provide code examples when helpful
3. Explain trade-offs between different approaches
4. Keep responses concise but complete"""

question = "What's the best way to handle configuration in a Python app?"

print("Expert Response:")
print(ask(question, system=expert_system))

### System Prompt Components

A good system prompt includes:

1. **Role/Identity**: Who is the assistant?
2. **Expertise**: What does it know?
3. **Behavior**: How should it respond?
4. **Constraints**: What should it avoid?
5. **Format**: How to structure responses?

---
## 5. Chain-of-Thought Prompting <a id='cot'></a>

**Chain-of-thought (CoT)** prompting encourages the model to think step by step.

This dramatically improves reasoning tasks.

In [ ]:
# GUIDED: Without chain-of-thought

math_problem = """If a store has 3 boxes of apples with 12 apples each, 
and they sell 15 apples, then receive a shipment of 2 boxes with 8 apples each,
how many apples do they have?"""

print("Without CoT:")
print(ask(math_problem, temperature=0))

In [ ]:
# GUIDED: With chain-of-thought

cot_prompt = f"""{math_problem}

Think through this step by step:
1. Calculate the starting number of apples
2. Subtract the apples sold
3. Add the new shipment
4. Give the final answer

Show your work:"""

print("With CoT:")
print(ask(cot_prompt, temperature=0))

In [ ]:
# GUIDED: Simple CoT trigger

# Just adding "Let's think step by step" can help!
simple_cot = f"""{math_problem}

Let's think step by step."""

print("Simple CoT trigger:")
print(ask(simple_cot, temperature=0))

### When to Use Chain-of-Thought

- **Math problems**: Multi-step calculations
- **Logic puzzles**: Reasoning through constraints
- **Complex analysis**: Breaking down problems
- **Decision making**: Weighing options
- **Debugging**: Tracing through code logic

---
## 6. Prompt Templates <a id='templates'></a>

Create reusable prompt templates for consistent results.

In [ ]:
# GUIDED: Create a prompt template class

class PromptTemplate:
    """A reusable prompt template with variable substitution."""
    
    def __init__(self, template: str):
        self.template = template
    
    def format(self, **kwargs) -> str:
        """Fill in the template with provided values."""
        return self.template.format(**kwargs)
    
    def __repr__(self):
        return f"PromptTemplate({self.template[:50]}...)"


# Create a code review template
code_review_template = PromptTemplate("""
Review this {language} code and provide feedback:

```{language}
{code}
```

Focus on:
1. Code correctness
2. Best practices
3. Potential improvements

Provide a rating from 1-10 and explain your reasoning.
""")

# Use the template
prompt = code_review_template.format(
    language="python",
    code="def add(a, b): return a + b"
)

print("Generated Prompt:")
print(prompt)

In [ ]:
# GUIDED: Library of prompt templates

PROMPT_LIBRARY = {
    "summarize": PromptTemplate(
        "Summarize the following text in {num_sentences} sentences:\n\n{text}"
    ),
    
    "translate": PromptTemplate(
        "Translate the following {source_lang} text to {target_lang}:\n\n{text}"
    ),
    
    "explain_code": PromptTemplate(
        """Explain this {language} code to a {audience}:

```{language}
{code}
```

Focus on what the code does, not how it works syntactically."""
    ),
    
    "debug": PromptTemplate(
        """Debug this {language} code:

```{language}
{code}
```

Error message: {error}

Identify the bug and provide a fix."""
    )
}

# Use a template from the library
prompt = PROMPT_LIBRARY["explain_code"].format(
    language="python",
    audience="beginner programmer",
    code="result = [x**2 for x in range(10) if x % 2 == 0]"
)

print("Code Explanation:")
print(ask(prompt))

---
## 7. Exercises <a id='exercises'></a>

Practice your prompt engineering skills!

### Exercise 1: Few-Shot Classifier

Create a few-shot prompt that classifies customer support tickets into categories:
- BILLING
- TECHNICAL
- GENERAL

In [ ]:
# TODO: Create a few-shot prompt for ticket classification
# Include 2-3 examples, then classify: "My payment didn't go through yesterday"

# Your code here:


### Exercise 2: System Prompt Design

Design a system prompt for an AI coding tutor that:
- Teaches beginners
- Uses encouraging language
- Gives hints instead of full solutions
- Asks follow-up questions

In [ ]:
# TODO: Design a system prompt for a coding tutor
# Then test it with a beginner question like "How do I create a list in Python?"

# Your code here:


### Exercise 3: Chain-of-Thought Problem

Use CoT prompting to solve:
"A train travels 120km in 2 hours. If it needs to travel 300km total and has already traveled for 2.5 hours at the same speed, how much longer will it take?"

In [ ]:
# TODO: Use chain-of-thought to solve the train problem

# Your code here:


---
## 8. Checkpoint <a id='checkpoint'></a>

Before moving on, verify:

- [ ] You understand zero-shot vs few-shot prompting
- [ ] You can write effective system prompts
- [ ] You know when and how to use chain-of-thought
- [ ] You created at least one prompt template
- [ ] You completed at least 2 exercises

### Next Steps

In the next notebook, we'll learn about **Tool Use Fundamentals** - giving LLMs the ability to use external tools and APIs!

---
## Summary

**Key Techniques:**

| Technique | Use Case |
|-----------|----------|
| Zero-shot | Common tasks with clear instructions |
| Few-shot | Custom formats, specific patterns |
| System prompts | Consistent behavior and personality |
| Chain-of-thought | Reasoning and complex problems |
| Templates | Reusable, consistent prompts |

**Remember:**
- Be specific and provide context
- Use examples when the task is non-standard
- For reasoning tasks, ask the model to think step by step
- Create templates for repeated use cases